In [129]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np

In [128]:
def resetParameters():
    global timePerWord
    global retrievalTimeWord
    global retrievalTimeSentence
    global steeringUpdateTime
    global startingPositionInLane
    global gaussDeviateMean
    global gaussDeviateSD
    global gaussDriveNoiseMean
    global gaussDriveNoiseSD
    global timeStepPerDriftUpdate
    global maxLateralVelocity
    global minLateralVelocity
    global startvelocity
    global wordsPerMinuteMean
    global wordsPerMinuteSD
    global carPos

    carPos = 0

    timePerWord = 0  ### ms

    retrievalTimeWord = 200   #ms
    retrievalTimeSentence = 300 #ms

    steeringUpdateTime = 250    #in ms
    startingPositionInLane = 0.27 			#assume that car starts already away from lane centre (in meters)


    gaussDeviateMean = 0
    gaussDeviateSD = 0.13 ##in meter/sec
    gaussDriveNoiseMean = 0
    gaussDriveNoiseSD = 0.1	#in meter/sec
    timeStepPerDriftUpdate = 50 ### msec: what is the time interval between two updates of lateral position?
    maxLateralVelocity = 1.7	# in m/s: maximum lateral velocity: what is the maximum that you can steer?
    minLateralVelocity = -1* maxLateralVelocity
    startvelocity = 0 	#a global parameter used to store the lateral velocity of the car
    wordsPerMinuteMean = 39.33
    wordsPerMinuteSD = 10.3

In [130]:
##calculates if the car is not accelerating (m/s) more than it should (maxLateralVelocity) or less than it should (minLateralVelocity)  (done for a vector of numbers)
def velocityCheckForVectors(velocityVectors):
    global maxLateralVelocity
    global minLateralVelocity

    velocityVectorsLoc = velocityVectors

    if (type(velocityVectorsLoc) is list):
            ### this can be done faster with for example numpy functions
        velocityVectorsLoc = velocityVectors
        for i in range(len(velocityVectorsLoc)):
            if(velocityVectorsLoc[i]>1.7):
                velocityVectorsLoc[i] = 1.7
            elif (velocityVectorsLoc[i] < -1.7):
                velocityVectorsLoc[i] = -1.7
    else:
        if(velocityVectorsLoc > 1.7):
            velocityVectorsLoc = 1.7
        elif (velocityVectorsLoc < -1.7):
            velocityVectorsLoc = -1.7

    return velocityVectorsLoc  ### in m/s

In [131]:
## Function to determine lateral velocity (controlled with steering wheel) based on where car is currently positioned. See Janssen & Brumby (2010) for more detailed explanation.
## Lateral velocity update depends on current position in lane. Intuition behind function: the further away you are, the stronger the correction will be that a human makes
def vehicleUpdateActiveSteering(LD):

	latVel = 0.2617 * LD*LD + 0.0233 * LD - 0.022
	returnValue = velocityCheckForVectors(latVel)
	return returnValue ### in m/s

In [132]:
from scipy.stats import norm

def normal_sample(mean,std):
    #x = 1.96  # value at which to evaluate the CDF
    return np.random.normal(loc=mean, scale=std, size=1)

In [133]:
def vehicleDrift():
    driftSpeed = normal_sample(gaussDeviateMean,gaussDeviateSD)
    carPos += velocityCheckForVectors(driftSpeed)

In [135]:
### function to update lateral deviation in cases where the driver is NOT steering actively (when they are distracted by typing for example). Draw a value from a random distribution. This can be added to the position where the car is already.
import math

def vehicleUpdateWord(sentence,word,nrSteeringMovementsWhenSteering):
    trailTime = 0

    if word == 0: trailTime += retrievalTimeSentence
    trailTime += retrievalTimeWord
    trailTime += timePerWord

    driftUpdates = math.floor(50 / trailTime)

    for dr in range(driftUpdates):
        vehicleDrift()
        locPos.append(carPos)
        locColor.append("red")
        locTime = properTime
        properTime += 50

    for steering in range(nrSteeringMovementsWhenSteering):
        trailTime += 250
        carPos += vehicleUpdateActiveSteering(carPos)
        locPos.append(carPos)
        locColor.append("blue")
        locTime = properTime
        properTime += 250

    return(trailTime)


In [125]:
### Function to run a trial. Needs to be defined by students (section 2 and 3 of assignment)

def runTrial(nrWordsPerSentence =5,nrSentences=3,nrSteeringMovementsWhenSteering=2, interleaving="word"):
    resetParameters()
    locPos = []
    locTime = []
    locColor = []

    totalTrailTime = 0
    properTime = 0


    if interleaving == "word":
        carPos = startingPositionInLane

        wordsPerMinute = normal_sample(wordsPerMinuteMean,wordsPerMinuteSD)
        timePerWord = ( 60 / wordsPerMinute ) * 1000

        for sentence in range(nrSentences):
            for word in range(nrWordsPerSentence):
                totalTrailTime += vehicleUpdateWord(sentence,word,nrSteeringMovementsWhenSteering)
    else:
        print("okay")

    #if interleaving == "sentence":

In [126]:
### function to run multiple simulations. Needs to be defined by students (section 3 of assignment)
def runSimulations(nrSims = 100):
    print("hello world")

In [136]:
runTrial()

UnboundLocalError: local variable 'carPos' referenced before assignment